# 03a - Record File Input Plot

In [1]:
import sys
!{sys.executable} -m pip install --upgrade pip -q
!{sys.executable} -m pip install pandas -q
!{sys.executable}  -m pip install numpy -q
!{sys.executable}  -m pip install matplotlib -q
!{sys.executable}   -m pip install scipy -q
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install psutil

In [2]:
#notes:
#next params Gather, Order, Research
create_posdf = False
#todos: calculate how many matches start in not diagonal opposing quadrants -ggf find another way to generalize

In [3]:
import numpy as np
import pandas as pd
import json
import pickle
import numpy.random
import matplotlib.pyplot as plt
from copy import deepcopy
from datetime import datetime 
from datetime import timedelta
from datetime import time
from matplotlib import cm
psutil
import gc


plt.rcParams["figure.figsize"] = (15,15)

psutil.virtual_memory()

svmem(total=540679213056, available=202066710528, percent=62.6, used=334329839616, free=204628910080, active=19713208320, inactive=305890889728, buffers=42250240, cached=1678213120, shared=259538944, slab=8236654592)

In [4]:
def print_txt(msg: str):
    #print(msg, end = '\r')
    text_file = open("data/scraped_matches/Output_inputplot.txt", "w")
    text_file.write(msg)
    text_file.close()
    
def print_txt_global(msg: str):
    print(msg)
    text_file = open("data/scraped_matches/Output_inputplot_global.txt", "w")
    text_file.write(msg)
    text_file.close()

In [5]:
psutil.virtual_memory()

svmem(total=540679213056, available=202056032256, percent=62.6, used=334341013504, free=204618727424, active=19713126400, inactive=305900318720, buffers=42266624, cached=1677205504, shared=259538944, slab=8238338048)

In [6]:
## def functions
def get_tile_value(tile_values: dict): 
    value = 0
    
    #for now zero for equal values
    if tile_values[1] > tile_values[2]:
        value = tile_values[1]
    elif tile_values[1] < tile_values[2]:
        value = -tile_values[2]
    
    return value

def get_result_matrix(control_matrix: list, distance_factor = 0.9, exploration_distance = 10, map_size =121):
    """ Creates a plotable result matrix based on a given control matrix with 3 values per coordinate"""
    #create result matrix
    result_matrix = []
    
    for x in range(map_size):
        y_list = []
        for y in range(map_size):
            y_list.append(get_tile_value(control_matrix[x][y]))
        result_matrix.append(y_list)

    result_matrix_initial = deepcopy(result_matrix)  
    
    moves = [i for i in range(-exploration_distance,exploration_distance+1)]
    
    
    #calculate values for tiles around 
    if False:
        for x in range(map_size):
            for y in range(map_size):
                if result_matrix_initial[x][y] == 0:
                    val_sum = 0
                    val_count = 0

                    for x_move in moves:
                        for y_move in moves:
                            try:
                                distance = abs(x_move) + abs(y_move)
                                diff = (result_matrix_initial[x + x_move][y + y_move] * pow(distance_factor,distance))

                                if diff != 0:
                                    val_count += 1
                                    val_sum += diff
                            except: 
                                pass 

                    try:
                        #print(result_matrix_initial[x][y])
                        result_matrix[x][y] = val_sum/val_count
                        #print(result_matrix_initial[x][y])

                    except:
                        pass
                    
            
    return result_matrix



def add_minutes(old_time: datetime.time, minutes: int):
    """ adds minutes to a given datetime.time"""
    hour = old_time.hour
    minute = old_time.minute
    second = old_time.second
    
    minute += minutes
    
    if minute >= 60:
        minute -= 60
        hour += 1
    
    t_new = time(hour = hour, minute = minute, second = second)
    return t_new

def create_control_matrix(view: pd.DataFrame, inf_dist = 5, map_size = 121):
    """ return mapcontrol matrix from view""" 
    control_matrix = {}
    #first round add together all values on tiles
    #create inital matrix with values for 0 control 
    for x in range(map_size):
        control_matrix[x]= {}
        for y in range(map_size):
            control_matrix[x][y] = {0: 0, 1: 0, 2: 0}
            


    #fill with number of actions on that given spot
    for index, action in view.iterrows():
        control_matrix[int(action['x_pos'])][int(action['y_pos'])][action['acting_player']] += 1


    #approach 1 normalize -> approach 2 would be to keep values in relation
    for x in range(map_size):
        for y in range(map_size):
            val_count = 0
            #count total values
            for index in control_matrix[x][y]:
                val_count += control_matrix[x][y][index]
            if False:
                #now normalize
                if val_count > 0:
                    for index in control_matrix[x][y]:
                        control_matrix[x][y][index] = control_matrix[x][y][index]/val_count
    return control_matrix

In [7]:
def create_plot_from_df(matches_df):
    import psutil
    temp_memory_dev = []
    
    #convert df into dict
    matches = matches_df.set_index('match_id').to_dict("index")

    ############################## read data from inputs ##############################
    error_count = 0
    tb_del = []
    
    #load pkl files
    for index, match in enumerate(matches):
        try:
            with open(matches[match]['input_fn'], 'rb') as f:
                matches[match]['inputs'] = pickle.load(f)
        except:
            error_count += 1
            tb_del.append(match)

        if index % 100 == 0:
            print("Imported ", index, " / ", len(matches), " - error on :", error_count,  end='\r')

    for match in tb_del:
      del matches[match]
    
    #gc.collect()
    print("Deleted: ",len(tb_del), " - Remaining: ", len(matches))
    

    ############################## create posdf ##############################

    #read in input data from matplotlib import cm
    count = 0
    pos_inputs = {}
    types = {}
    
    for index, match in enumerate(matches):
        
        #create single dict per match
        with open(matches[match]['input_fn'], 'rb') as f:
                matches[match]['inputs'] = pickle.load(f)
        
        for input in matches[match]['inputs']:
            #determine acting player
            if matches[match]['inputs'][input]['player'] ==  'p1':
                acting_player = 1#matches[match]['p1_name']
            elif matches[match]['inputs'][input]['player'] ==  'p2':
                acting_player = 2#matches[match]['p2_name']
            else: acting_player = 0

            ##obsolete
            #print(acting_player, matches[match]['inputs'][input]['x_pos'])
            this_type = matches[match]['inputs'][input]['type']

            if this_type not in types:
                types[this_type] = 0
            else:
                types[this_type] += 1

            #only read posistion and player related inputs
            if matches[match]['inputs'][input]['x_pos'] != None and acting_player != 0:
                count += 1
                row_id = str(match)+"-"+str(input)
                pos_inputs[row_id] = {}
                pos_inputs[row_id]['match_id'] = match
                pos_inputs[row_id]['move_id'] = input

                #enrich time data
                try:
                    pos_inputs[row_id]['time'] = datetime.strptime(matches[match]['inputs'][input]['timestamp'],'%H:%M:%S.%f').time()
                except:
                    pos_inputs[row_id]['time'] = datetime.strptime(matches[match]['inputs'][input]['timestamp'],'%H:%M:%S').time()

                #read player data
                pos_inputs[row_id]['acting_player'] = acting_player

                #read redundant data from matches into every row: 
                for key in matches[match].keys():
                    pos_inputs[row_id][key] = matches[match][key]

                #add move data
                for key in matches[match]['inputs'][input]:
                    pos_inputs[row_id][key] = matches[match]['inputs'][input][key]
        
        #clear memory
        del matches[match]['inputs']
        
        if index % 1000 == 0:
            temp_pos_inputs_df = pd.DataFrame.from_dict(pos_inputs,orient="index")

            if index > 1000:
                    pos_inputs_df = pd.concat([pos_inputs_df, temp_pos_inputs_df])
            else:
                #first run
                pos_inputs_df = temp_pos_inputs_df

            print_txt(f'Concat after {index} matches - currently {pos_inputs_df.shape}')

            #reset pos inputs
            pos_inputs = {}
            gc.collect()
        
        

        print("Read Data from Match  ", index, " / ", len(matches),  end='\r')

    print("Read Data from Match  ", index, " / ", len(matches))
    print("Found ", len(pos_inputs), " Inputs")
    print("Start creating df from dict")

    #final append
    temp_pos_inputs_df = pd.DataFrame.from_dict(pos_inputs,orient="index")               
    pos_inputs_df = pd.concat([pos_inputs_df, temp_pos_inputs_df])


    #final persistation
    output = open('data/scraped_matches/pos_inputs_df.pkl' , 'wb')
    pickle.dump(pos_inputs_df, output)
    output.close()

    print_txt(f'Concat after {len(matches)} matches - currentetly {pos_inputs_df.shape}')
    print(types)


    print(f'Shape pos_df: {pos_inputs_df.shape}, memory: {str(psutil.virtual_memory()[2])} %')

    ############################## create plots ##############################
    dir_path = 'data/minn/unfoldered_layers/'
    actions = ["Build","Target","Move"]
    map_size = 121
    errors = 0
    error_list = []

    import psutil
    psutil.virtual_memory()

    for counter, match in enumerate(matches):

        try:
            pos_inputs_match_df = pos_inputs_df[pos_inputs_df['match_id'] == match]
            #create views for the different actions
            df_views = {}
            views = []
            for action in actions:
                df_views[action] = pos_inputs_match_df[pos_inputs_match_df['type'] == action]
                df_views[action].name = action
                views.append(df_views[action])

            #create plots for the different views
            for view in views:
                #get matrix
                result_matrix = get_result_matrix(create_control_matrix(view, map_size = map_size),map_size = map_size)
                #plot
                figure, axes = plt.subplots()


                axes.pcolormesh(result_matrix, cmap='RdYlGn', vmin=-1, vmax=1)

                plt.pcolormesh(result_matrix, cmap='RdYlGn', vmin=-1, vmax=1)
                plt.axis([0,map_size,0,map_size])

                file_path = dir_path + str(match) +'_'+ view.name +'_layer.png'

                #correct format
                axes.set_axis_off()
                figure.set_size_inches(12, 12)

                plt.savefig(file_path,dpi = 10)
                plt.close(figure)
                
                del result_matrix
                del figure
                del axes
                
                #plt.show()
                #plt.rcParams["figure.figsize"] = (15,15)


            #delete pos inputs for match
            pos_inputs_df = pos_inputs_df[pos_inputs_df['match_id'] != match]
            del pos_inputs_match_df

        except Exception as e:
            print(e)
            errors +=1
            error_list.append(match)
        
        
        msg = f'Saved {counter+1}/{len(matches)} matches - last one was: {match} - error on: {errors} - memory: {str(psutil.virtual_memory()[2])} %'
        print(msg, end='\r')

        if counter % 100 == 0:
            temp_memory_dev.append(' ' + str(psutil.virtual_memory()[2]) + '%')
            print_txt(msg + str(temp_memory_dev))
        
        
    ############################## del everything ##############################
    del matches
    del pos_inputs_df
    del temp_pos_inputs_df
    del df_views
    del views
    del output
    gc.collect()
    #del matches_df
    #for view in views:
        #del view
    #del pos_inputs_df
    #del pos_inputs_match_df
    #del temp_pos_inputs_df
    #gc.collect()
        

In [8]:
#load matches
matches_df = pd.read_csv('data/scraped_recordfile_matches.csv')
matches_df.head()

<ipython-input-8-9204aad189b1>:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  matches_df = pd.read_csv('data/scraped_recordfile_matches.csv')


,match_id,recordfile,input_fn,map,map_size,duration,p1_name,p2_name,p1_civ,p2_civ,p1_xpos,p2_xpos,p1_ypos,p2_ypos,winner,gaia_fn
0,87925986,AgeIIDE_Replay_87925986.aoe2record,data/scraped_matches/inputs/87925986.pkl,Serengeti,Tiny,3003,48mb,Sat Thien Mach,Incas,Vikings,84.0,23.0,98.0,31.0,0,NaN
1,109059887,AgeIIDE_Replay_109059887.aoe2record,data/scraped_matches/inputs/109059887.pkl,Arabia,Tiny,3472,Ralf,StormX,Berbers,Lithuanians,92.0,22.0,83.0,75.0,1,NaN
2,132137828,AgeIIDE_Replay_132137828.aoe2record,data/scraped_matches/inputs/132137828.pkl,Arabia,Tiny,746,SaulGoodman,[Tkilla] I'm A.I.,Franks,Mongols,44.0,62.0,21.0,99.0,1,NaN
3,63169375,AgeIIDE_Replay_63169375.aoe2record,data/scraped_matches/inputs/63169375.pkl,Arabia,Tiny,1866,Oblivious White Eel,Shotgun Roddy,Ethiopians,Koreans,20.0,98.0,62.0,51.0,1,NaN
4,77688193,AgeIIDE_Replay_77688193.aoe2record,data/scraped_matches/inputs/77688193.pkl,Arabia,Tiny,5036,Sinede,TheEskimo,Vietnamese,Malay,44.0,50.0,96.0,21.0,0,NaN


In [9]:
#find already scraped matches for parallel download
"""
plotted_matches = []

for file in os.listdir(zip_path):
    filename = os.fsdecode(file)

    match_id = filename.partition("=")[2].partition("&")[0]
    scraped_matches.append(match_id)
    
"""

'\nplotted_matches = []\n\nfor file in os.listdir(zip_path):\n    filename = os.fsdecode(file)\n\n    match_id = filename.partition("=")[2].partition("&")[0]\n    scraped_matches.append(match_id)\n    \n'

In [10]:
size = 2000
total = matches_df.shape[0]
position = 74000

In [ ]:
memory_dev = []

while position < total:
    #create temp_df
    temp_matches_df = matches_df[matches_df.index < position + size]
    temp_matches_df = temp_matches_df[temp_matches_df.index >= position] 
    
    memory_dev.append(' ' + str(psutil.virtual_memory()[2]) + '%')
    print_txt_global(f'Plotting matches {position} - {position + size} of {total} matches - Memory: {psutil.virtual_memory()[2]} % -Dev: {memory_dev}')
    create_plot_from_df(temp_matches_df)
    gc.collect()
    
    position += size

Plotting matches 74000 - 76000 of 80104 matches - Memory: 62.6 % -Dev: [' 62.6%']
Deleted:  0  - Remaining:  2000r on : 0
